# Instructions

## 1. Device Setup

In [ ]:
from laboneq.simple import SHFQC
from laboneq.dsl.device import DeviceSetup, create_connection
import numpy as np
from copy import deepcopy

host = "10.172.11.217"
#instantiate ad device setup object
setup = DeviceSetup(uid="test_setup")

#add a dataserver
setup.add_dataserver(host=host, port="8004",uid="test_server")

setup.add_instruments(SHFQC(uid="test_shfqc",server_uid="test_server", address="DEV12576", device_options="SHFQC/PLUS/QC6CH"))

setup.instruments

In [ ]:
for ch_idx, q_idx in enumerate([15,16,17,18,19]):# enumerate 안에 len(list) 강제하고싶음
    #QA
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/measure", ports="QACHANNELS/0/OUTPUT")) #signal_type="iq"
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/acquire", ports="QACHANNELS/0/INPUT" )) #signal_type="acquire"
    #SG
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_ef", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_cr", ports=f"SGCHANNELS/{ch_idx}/OUTPUT"))#signal_type="iq"


In [ ]:
from operations import TransmonOperations
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters


from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform
from laboneq.dsl.quantum.qpu import QuantumElement


_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
qubits = list(_loaded_qubits.values())
################################################################################################
qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
qpu.topology.add_edge("bus", "q16", "q17")
qpu.topology.add_edge("bus", "q17", "q18")
qpu.topology.add_edge("bus", "q18", "q19")
qpu.topology.plot()

# Create a demonstration QuantumPlatform for a transmon QPU:
quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)


from laboneq.simple import workflow
from pathlib import Path


# Two Types of store in workflow!
 
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().deactivate()
#workflow.logbook.LogbookStore().deactivate()


### Initial Setting

In [ ]:
# for _, qubit in enumerate(qubits):
#     ##################LOCAL OSCILLATOR FREQUENCY AND CHANNEL POWER ###################################
#     qubit.parameters.drive_lo_frequency = 4.8e9
#     qubit.parameters.readout_lo_frequency = 7.0e9
#     qubit.parameters.readout_range_in =  0
#     qubit.parameters.readout_range_out = -25
#     qubit.parameters.drive_range = 0 
    
    
    
#     ############READOUT PARAMETERS###################################################################
#     qubit.parameters.readout_length - 2.0e-6
#     qubit.parameters.readout_pulse['function'] = 'GaussianSquare'
#     qubit.parameters.readout_pulse['sigma'] = 0.1
#     qubit.parameters.readout_pulse['width'] = None
#     qubit.parameters.readout_pulse['risefall_sigma_ratio'] = 3.0
#     qubit.parameters.readout_amplitude = 1.0  #WARNING! 이거 SWEEP 할때 오버라이딩이 안되고 여기서 SCALING 됨
    
#     qubit.parameters.reset_delay_length = 200e-6
#     ###########SPECTROSCOPY PARAMETERS###############################################################
#     # Pulsed qubit spectroscopy drive ,  
#     qubit.parameters.spectroscopy_pulse['function'] = 'GaussianSquare'
#     qubit.parameters.spectroscopy_length = 5.0e-6
#     qubit.parameters.spectroscopy_pulse['sigma'] = 0.2
#     qubit.parameters.spectroscopy_pulse['width'] = None
#     qubit.parameters.spectroscopy_pulse['risefall_sigma_ratio'] = 3.0
#     qubit.parameters.spectroscopy_amplitude=0.01 
    
    
  

# qubits[3].parameters.readout_amplitude = 0.5
# qubits[4].parameters.readout_amplitude = 0.5
# qubits[2].parameters.readout_amplitude = 0.5


In [ ]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True)

# Run Experiments
BaseExperimentOptions Default setting
- `count` 1024
- `AveragingMode.CYCLIC`
- `AcquisitionType.INTEGRATION`
- `RepetitionMode.FASTEST`  

`WorkflowOptions` and `TaskOptions`  
Here Tuneupworkflowoptio is workflowoption and DirectCR~ is taskoption

In [ ]:
from experiments import direct_cr_hamiltonian_tomography
from experiments.direct_cr_hamiltonian_tomography import DirectCRHamiltonianTomographyOptions
############################################################
options = direct_cr_hamiltonian_tomography.experiment_workflow.options()
options.do_analysis(False)
options.update(False)
options.count(1024)
options.do_plotting(False)
#print(workflow.show_fields(options))
######################################################################
hamil_tomo = direct_cr_hamiltonian_tomography.experiment_workflow(
    session=session,
    qpu = qpu,
    ctrl = qubits[2],
    targ = qubits[3],
    amplitudes = np.asarray([0.1]),
    lengths = np.arange(200e-9, 600e-9, 10e-9),
    options=options,
)
hamil_tomo_result = hamil_tomo.run()

# Fitting

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fit_results[0.1]['g'].best_fit)
plt.plot(fit_results[0.1]['g'].init_fit)

In [ ]:
fit_results[0.1]['e']


pz=fit_results[0.1]['e'].best_values['pz']
py=fit_results[0.1]['e'].best_values['py']
px=fit_results[0.1]['e'].best_values['px']

#omega = np.sqrt(pz**2 + px**2 + py**2)
omega = np.average([px,py,pz])

np.arcsin(pz/omega)


In [ ]:
from analysis.hamiltonian_tomography import fit_data, process_data, plot_trajectory
qubit = qubits[3]
amplitudes = np.asarray([0.1])
lengths = np.arange(200e-9, 600e-9, 10e-9)
processed_data_dict = process_data(qubit, hamil_tomo_result.output, lengths, amplitudes)
fit_results = fit_data(qubit,processed_data_dict, lengths, amplitudes)
figures = plot_trajectory(qubit, processed_data_dict, fit_results, lengths, amplitudes)

In [ ]:
fit_results[0.1]['e']

In [ ]:
g_val = fit_results[0.1]['g'].best_values
e_val = fit_results[0.1]['e'].best_values

delta0 = g_val['pz']/(2*np.pi*1e6)
omega0_x = g_val['px']/(2*np.pi*1e6)
omega0_y = g_val['py']/(2*np.pi*1e6)

delta1 = e_val['pz']/(2*np.pi*1e6)
omega1_x = e_val['px']/(2*np.pi*1e6)
omega1_y = e_val['py']/(2*np.pi*1e6)

IX = 0.5 * (omega0_x + omega1_x) 
IY = 0.5 * (omega0_y + omega1_y)
IZ = 0.5 * (delta0 + delta1)

ZX = 0.5 * (omega0_x - omega1_x)
ZY = 0.5 * (omega0_y - omega1_y)
ZZ = 0.5 * (delta0 - delta1)

print(IX,IY,IZ,ZX,ZY,ZZ)



print(np.sqrt(IX**2 + IY**2))
print(np.sqrt(ZX**2+ZY**2))

-np.arctan2(IX,IX)

# Compilation Check

In [ ]:
options = direct_cr_hamiltonian_tomography.experiment_workflow.options()


hamil_tomo_compiled = session.compile(
    direct_cr_hamiltonian_tomography.create_experiment(
        qpu=qpu,
        ctrl=qubits[2],
        targ=qubits[3],
        amplitudes=np.asarray([0.4]),
        lengths=np.asarray([200e-9, 300e-9]),
    ))


from laboneq.simple import show_pulse_sheet
from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
show_pulse_sheet("first_comp", hamil_tomo_compiled, interactive=True)
plot_simulation(hamil_tomo_compiled, length=2e-6)

# Save Qubit Parameters

In [ ]:
# from helper import load_qubit_parameters, save_qubit_parameters
# save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="kaist")